To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Llama-3 8b is trained on a crazy 15 trillion tokens! Llama-2 was 2 trillion.**

Use our [Llama-3 8b Instruct](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing) notebook for conversational style finetunes.

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install xformers

MODEL_VERSION ="mOptJson"
directory = "/content/drive/MyDrive/vibviz/lora-alpaca/" + MODEL_VERSION + "/"

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton


MODEL_VERSION ="mOptJson"
directory = "/content/drive/MyDrive/vibviz/lora-alpaca/" + MODEL_VERSION + "/"

In [8]:
MODEL_VERSION = "mOptJsonTwo"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [4]:
# Helpers

############################################################################################
# Falls JSON nicht fertig generiert wurde, wird json beim letzten vollständigen Block beendet.
# Gibt Json Objekt zurück
############################################################################################
def fixUnfinishedJson(output):

    import json
    try:
        ojson = json.loads(output)
        return ojson
    except json.decoder.JSONDecodeError:
        lastValidIndex = output.rfind("""}},""")
        print(lastValidIndex)
        lastValidIndex += len("""}},""")-1

        output = output[:lastValidIndex]
        output += """]}"""
        print(output)
        return json.loads(output)

############################################################################################
# Filtert aus gesamter KI-Ausgabe (string) die Antowort raus
############################################################################################
def findResponse(output):
    response= "### Response:"
    response_index = output.find(response)
    response_index += len(response)
    output = output[response_index:]
    return output

import json
import os
############################################################################################
# Gibt die nächste Versionsnummer zurück
############################################################################################
def get_next_version(fullpath):
    """
    Finds the next version number
    """

    # Split the path into directory and filename
    directory, full_file_name = os.path.split(fullpath)


    # Check if the directory exists, if not create it and return 0
    if not os.path.exists(directory):
        os.makedirs(directory)

    from pathlib import Path
    file = Path(f"{directory}\\{ID}_.json")
    if file.exists():

        j = json.load(file.open("r"))
        j["v"] = j["v"] + 1
        json.dump(j, file.open("w"))
        return j["v"]
    else:
        j = {"v": 1}
        json.dump(j, file.open("w"))
        return j["v"]

############################################################################################
# Gibt Dateiname und Speicherpfad zurück für organisiertes Speichern der Ergebnisse
############################################################################################
def getFilePath(name="", ext =""):
    return f"""{directory}{ID}_v{VERSION}_{name}{ext}"""

############################################################################################
# Generiert Samples aus Json Objekt
############################################################################################
def generateSamples(hlabsJson):
        import numpy
        sample_rate = 16000
        audio_arr = []

        for block in hlabsJson["sequence"]:
            if block["type"] == "v": # wemm Sinuswelle

                duration_in_seconds = block["data"]["dur"] / 1000
                ampitude = block["data"]["amp"]
                fequencie = block["data"]["freq"]

                t = numpy.linspace(0, duration_in_seconds, int(sample_rate *duration_in_seconds), endpoint=False)  # Time array
                audio_arr.extend(ampitude * numpy.sin(2 * numpy.pi * fequencie * t))  # Sine wave formula

            else: # wenn Pause
                duration_in_seconds = block["data"]["dur"] / 1000
                audio_arr.extend(numpy.zeros(int(sample_rate * duration_in_seconds)))
        return numpy.float32(audio_arr)

############################################################################################
# Visualisiert die Ausgabe (Json Objekt)
############################################################################################
def plotOutput(ojson):
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches

    plt.figure().set_figwidth(20)

    amplitudes = []
    frequencies = []
    times = []
    times.append(0)
    for block in ojson["sequence"]:
        if block["type"] == "v":
            amplitudes.append(block["data"]["amp"])
            frequencies.append(block["data"]["freq"])
        else:
            amplitudes.append(0)
            frequencies.append(0)
            if len(times) > 0:
                plt.axvspan(times[-1], times[-1] + block["data"]["dur"], color="green", alpha=0.5)
            else:
                plt.axvspan(0, block["data"]["dur"], color="green", alpha=1)
        if len(times) == 0:
            times.append(block["data"]["dur"])
        else:
            times.append(times[-1] + block["data"]["dur"])
    times_block_mid = []
    times_block_mid.append(times[0] - 0 / 2)


    line1 =plt.hlines(amplitudes, times[:-1], times[1:], color="blue",label="Amplitude")
    plt.ylabel("Amplitude")
    plt.xlabel("Zeit in Millisekunden")
    plt.legend(loc="upper right")
    plt.twinx()
    line2=plt.hlines(frequencies, times[:-1], times[1:], color="red",label="Frequenz")
    plt.ylabel("Frequenz in Hertz")
    plt.legend(handles=[line1, line2])
    plt.savefig(getFilePath(name="Blocks",ext=".svg"))

############################################################################################
# Visualisiert Signal
############################################################################################
def plotOutputSignal(audio_arr):
    import numpy
    import matplotlib.pyplot as plt
    times = numpy.arange(0,len(audio_arr)/16,1/16)
    plt.figure().set_figwidth(20)
    plt.plot(times, audio_arr,label="Signal")
    plt.hlines([-1,1],0,times[-1],colors="red",label="Amplitudengrenzen")
    plt.xlabel("Zeit in Millisekunden")
    plt.legend()
    plt.savefig(getFilePath(name="Signal",ext=".svg"))

############################################################################################
# Speichert Signal als wav-Datei
############################################################################################
def saveAsWav(audio_arr, sr):
    import scipy.io.wavfile
    filename = getFilePath(name="",ext=".wav")
    scipy.io.wavfile.write(filename, sr, audio_arr)
    return filename


ID = "None" #Wiedererkennung des Prompts (z.B ID = ID des Datensatzes aus dem der Prompt stammt)
VERSION = -1 #wird durch get_next_version() gesetzt (pro generierter Ausgabe)


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128 #Die Dimension der LoRa Matrizen
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [9]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
csv_path = r"/content/drive/MyDrive/vibviz/vibrationAnnotations-July24th2016.csv"
json_path_train = r"/content/drive/MyDrive/vibviz/jsonOptimiert"
json_path_test = r"/content/drive/MyDrive/vibviz/jsonOptimiertTest"
json_path = json_path_train
naturalPromptsJson = r"/content/drive/MyDrive/vibviz/prompts.json"
fromJSON = True
fromCSV = False


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

import os
#Lädt json Datei und gibt Inhalt als string zurück
def readJson(file_name):
  file_path = os.path.join(json_path, file_name + ".json")
  if not os.path.isfile(file_path):
    raise Exception("File", file_path, "does not exists.")
  file = open(file_path,'r')
  content = file.read()
  file.close()
  return content


from datasets import load_dataset
# Datensätze mit fehlender JSON löschen
def delete_if_json_is_missing(dataset):
  keep_indices = []
  for i in range(len(dataset["id"])):
    file_path = os.path.join(json_path, dataset["id"][i] + ".json")

    if not os.path.isfile(file_path):
      print("Warning: File", file_path, "does not exists. Dataset is ignored. Idex:", i)
    else:
      keep_indices.append(i)
  return dataset.select(keep_indices)

def value_to_description(value):
 pass

#Formatiert die Daten für das fine tuning-> Rohdaten aus VibViz Datenbank
def vibviz_formatting_prompts_func(examples):
  instruction = "Generate an vibrotaktil pattern with following characteristics:" #Instruction verknüpt im Modell die Aufgabe mit der Eingabe
  texts = []
  for i in range(len(examples["id"])):
    input = ""
    if examples["sensationTags"][i] is not None:
      input = input + examples["sensationTags"][i] + ","
    if examples["emotionTags"][i] is not None:
      input = input + examples["emotionTags"][i] + ","
    if examples["metaphors"][i] is not None:
      input = input + examples["metaphors"][i] + ","
    if examples["usageExamples"][i] is not None:
      input = input + examples["usageExamples"][i]
    output = readJson(examples["id"][i])
    if output is not None:
      text = alpaca_prompt.format(instruction,input,output) + EOS_TOKEN
      texts.append(text)
  return {"text" : texts}

def vibviz_formatting_natural_prompts_func(examples):
  instruction = ""
  texts = []
  for i in range(len(examples["id"])):
    input = examples["1"][i]
    output = readJson(examples["id"][i])
    text = alpaca_prompt.format(instruction,input,output) + EOS_TOKEN
    texts.append(text)
  return {"text" : texts}



if fromCSV:
  #Daten laden
  dataset = load_dataset('csv', data_files=csv_path, split="train")
  #Datem mit fehlender json aussortieren
  dataset = delete_if_json_is_missing(dataset)

  #dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
  #Daten formatieren "mapen" für das Training
  dataset = dataset.map(vibviz_formatting_prompts_func, batched = True,)

if fromJSON:
  import json
  import pandas as pd
  import datasets
  #Trainingsdaten

  json_path = json_path_train
  with open(naturalPromptsJson) as f:
    data = json.load(f)
  df = pd.DataFrame.from_dict(data, orient='index')
  dataset = datasets.Dataset.from_pandas(df, split="train").rename_column("__index_level_0__","id")
  #Datem mit fehlender json aussortieren
  dataset = delete_if_json_is_missing(dataset)
  #Daten formatieren "mapen" für das Training
  dataset = dataset.map(vibviz_formatting_natural_prompts_func, batched = True)

  #Testdaten
  json_path = json_path_test
  with open(naturalPromptsJson) as f:
    data = json.load(f)
  df = pd.DataFrame.from_dict(data, orient='index')
  datasetTest = datasets.Dataset.from_pandas(df, split="test").rename_column("__index_level_0__","id")
  #Datem mit fehlender json aussortieren
  datasetTest = delete_if_json_is_missing(datasetTest)
  #Daten formatieren "mapen" für das Training
  datasetTest = datasetTest.map(vibviz_formatting_natural_prompts_func, batched = True)

# Entimmt aus den Trainingsdaten zufällige Testdatem
if False:
  dataset = dataset.train_test_split(test_size = 0.03, shuffle = True, seed = 3407)



Map:   0%|          | 0/115 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = datasetTest,
    eval_packing=False,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        warmup_steps = 5,
        num_train_epochs=2,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        eval_steps=10,
        evaluation_strategy = "steps",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/115 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
9.646 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 115 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 1
\        /    Total batch size = 2 | Total steps = 116
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
10,0.933600,0.515466
20,0.576600,0.448030
30,0.613500,0.415651
40,0.495400,0.407695
50,0.538800,0.406786
60,0.437700,0.395885
70,0.438400,0.394041
80,0.406300,0.393714
90,0.416900,0.388604
100,0.396100,0.382673


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

476.2773 seconds used for training.
7.94 minutes used for training.
Peak reserved memory = 7.535 GB.
Peak reserved memory for training = 1.941 GB.
Peak reserved memory % of max memory = 51.092 %.
Peak reserved memory for training % of max memory = 13.161 %.


## Test

In [13]:
#Ursprünglicher plan 10 Datensätze 10 tests ca. 8 bis 16 1/2h Uff shit noo
#Neuer Plan 5 Datensätze 5 Teste ca. 2h-4h mal schauen + 1h Training uff nee

#durschschitt 4:15 pro Generierung
#durchschnitt nach Optimierung 1:15 pro Generierung

import json

testsPerDataset = 5 #dauer ca 1 1/2 h

def findPrompt(data):
  response= "### Response:"
  input="### Input:"
  response_index = data.find(response)
  input_index = data.find(input)

  input_index += len(input)
  return data[input_index:response_index]



FastLanguageModel.for_inference(model) # Enable native 2x faster inference

tests = []

for y, data in enumerate(datasetTest):

  ID = f"testNr{y}"

  print(f"Testdaten: {y+1}/{len(datasetTest)}")
  prompt = findPrompt(data["text"])
  tests.append({"prompt":prompt,
               "referenceData":data["text"],
               "generatedData": []
               })
  failedGenerationCount = 0

  #Anzahl der Tests je Testdatensatz
  for i in range(testsPerDataset):

   failedGeneration = True

   while failedGeneration:
    failedGeneration = False
    VERSION = get_next_version(getFilePath())
    print(f"Test: {i+1}/5")

    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "", # instruction
            prompt, # input #v-09-12-8-27
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_length = 8000,max_time=360,use_cache = True) #LLama 3 unterstützt max 8000 Tokens #Abbrechen nach 10 Minuten
    tokenizer.batch_decode(outputs)
    outstring = tokenizer.decode(outputs[0], skip_special_tokens=True)

    #Versuch ausgabe in Json Objekt umzuwandeln um ausgabe des LLM auf korrektheit zu prüfuen
    try:
      generated_text = findResponse(outstring) #Findet die Antwort in string
      oJson = fixUnfinishedJson(generated_text) # Fixt die Antwort falls sie nicht fertig generiert wurde
      json.dump(oJson, open(getFilePath(ext=".json"),"w"))

      with open(getFilePath(name="prompt",ext=".txt"),"w") as f:
        f.write(prompt)

    #Wenn llm fehler bei generierung eingebaut hat nochmal versuchen
    except json.decoder.JSONDecodeError:
      if failedGenerationCount < testsPerDataset: #Maximal x Wiederholungen pro Testdatensatz
        failedGeneration = True
        failedGenerationCount += 1
        print("Generation failed. Trying again.")
      else:
        print("Generation failed. Skipping.")
      print("gesamte Ausgabe:",outstring)
      continue
    tests[y]["generatedData"].append(outstring)

  tests[y]["failedGenerations"] = failedGenerationCount

#Ergebnisse speichern
json.dump(tests,open(getFilePath(name="tests",ext=".json"),"w"))

Testdaten: 1/4
Test: 1/5
Test: 2/5
9947

{"sequence": [{"type": "p", "data": {"dur": 68}}, {"type": "v", "data": {"amp": 0.09, "freq": 93, "form": "sine", "dur": 26}}, {"type": "p", "data": {"dur": 42}}, {"type": "v", "data": {"amp": 0.1, "freq": 93, "form": "sine", "dur": 26}}, {"type": "p", "data": {"dur": 42}}, {"type": "v", "data": {"amp": 0.09, "freq": 93, "form": "sine", "dur": 26}}, {"type": "p", "data": {"dur": 42}}, {"type": "v", "data": {"amp": 0.1, "freq": 93, "form": "sine", "dur": 26}}, {"type": "p", "data": {"dur": 42}}, {"type": "v", "data": {"amp": 0.09, "freq": 93, "form": "sine", "dur": 26}}, {"type": "p", "data": {"dur": 42}}, {"type": "v", "data": {"amp": 0.1, "freq": 93, "form": "sine", "dur": 26}}, {"type": "p", "data": {"dur": 42}}, {"type": "v", "data": {"amp": 0.09, "freq": 93, "form": "sine", "dur": 26}}, {"type": "p", "data": {"dur": 42}}, {"type": "v", "data": {"amp": 0.1, "freq": 93, "form": "sine", "dur": 26}}, {"type": "p", "data": {"dur": 42}}, {"type": 

In [ ]:
#Test an Begriffen
#32 minuten und 15 sekunden - 1:38 pro generierung
FastLanguageModel.for_inference(model)
prompts = [
    "Generate a long vibration.",
    "Generate a short vibrotactile pattern.",
    "Generate a vibration that ramps up.",
    "Generate a vibrotactile pattern that ramps down.",
    "Create a rhythmic vibration.",
    "Create a vibration that feels like tapping.",
    "Create a vibrotactile pattern that feels like a heartbeat.",
    "Create a vibration that feels rough.",
    "Generate a short and rhythmic vibrotactile pattern that ramps down.",
    "Generate a long vibration that ramps up. It should feel like tapping. I want to use it as an alarm."
]

for prompt in prompts:
    VERSION = get_next_version(getFilePath())
    for i in range(2):
      print(f"Test: {i+1}/2")
      ID = i

      inputs = tokenizer(
      [
        alpaca_prompt.format(
            "", # instruction
            prompt,
            "", # output - leave this blank for generation!
        )
      ], return_tensors = "pt").to("cuda")
      outputs = model.generate(**inputs, max_length = 8000,max_time=360,use_cache = True) #LLama 3 unterstützt max 8000 Tokens #Abbrechen nach 10 Minuten
      tokenizer.batch_decode(outputs)
      outstring = tokenizer.decode(outputs[0], skip_special_tokens=True)

      #Versuch ausgabe in Json Objekt umzuwandeln um ausgabe des LLM auf korrektheit zu prüfuen
      try:
        generated_text = findResponse(outstring) #Findet die Antwort in string
        oJson = fixUnfinishedJson(generated_text) # Fixt die Antwort falls sie nicht fertig generiert wurde
        json.dump(oJson, open(getFilePath(ext=".json",name="testwithsentences"),"w"))

        with open(getFilePath(name="prompt",ext=".txt"),"w") as f:
          f.write(prompt)
        samples=generateSamples(oJson)
        plotOutput(oJson)
        plotOutputSignal(samples)
      except:
        print("Fehler beim Umwandeln in Json")


Test: 1/2
10193

{"sequence": [{"type": "p", "data": {"dur": 25}}, {"type": "v", "data": {"amp": 0.93, "freq": 100, "form": "sine", "dur": 44}}, {"type": "p", "data": {"dur": 25}}, {"type": "v", "data": {"amp": 0.93, "freq": 100, "form": "sine", "dur": 44}}, {"type": "p", "data": {"dur": 25}}, {"type": "v", "data": {"amp": 0.93, "freq": 100, "form": "sine", "dur": 44}}, {"type": "p", "data": {"dur": 25}}, {"type": "v", "data": {"amp": 0.93, "freq": 100, "form": "sine", "dur": 44}}, {"type": "p", "data": {"dur": 25}}, {"type": "v", "data": {"amp": 0.93, "freq": 100, "form": "sine", "dur": 44}}, {"type": "p", "data": {"dur": 25}}, {"type": "v", "data": {"amp": 0.93, "freq": 100, "form": "sine", "dur": 44}}, {"type": "p", "data": {"dur": 25}}, {"type": "v", "data": {"amp": 0.93, "freq": 100, "form": "sine", "dur": 44}}, {"type": "p", "data": {"dur": 25}}, {"type": "v", "data": {"amp": 0.93, "freq": 100, "form": "sine", "dur": 44}}, {"type": "p", "data": {"dur": 25}}, {"type": "v", "data":

<ipython-input-6-4a74ab3bb195>:34: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure().set_figwidth(20)


Test: 2/2
9875

{"sequence": [{"type": "p", "data": {"dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 28}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}, {"type": "v", "data": {"amp": 0.99, "freq": 138, "form": "sine", "dur": 27}}

## Save Model to google drive

In [ ]:
# prompt: Save model to google drive

# ## Save Model to google drive
output_dir = '/content/drive/MyDrive/vibviz/lora-alpaca/' + MODEL_VERSION
model.save_pretrained(output_dir)


## Load saved model

In [ ]:
max_seq_length = 8000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
  import torch
  from unsloth import FastLanguageModel
  model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/vibviz/lora-alpaca/" + MODEL_VERSION, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype ,
    load_in_4bit = load_in_4bit,
  )
  alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
pass

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

### JSON in Google Drive speichern

In [ ]:
import json
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True) #Ausgabe in String umwandeln
generated_text = findResponse(generated_text) #Findet die Antwort in string
oJson = fixUnfinishedJson(generated_text) # Fixt die Antwort falls sie nicht fertig generiert wurde

json.dump(oJson, open(getFilePath(ext=".json"),"w"))

with open(getFilePath(name="prompt",ext=".txt"),"w") as f:
  f.write(prompt)


## Visualisierung der Ausgabe

In [6]:
# Visualisierung


############################################################################################
# Generiert Samples aus Json Objekt
############################################################################################
def generateSamples(hlabsJson):
        import numpy
        sample_rate = 16000
        audio_arr = []

        for block in hlabsJson["sequence"]:
            if block["type"] == "v": # wemm Sinuswelle

                duration_in_seconds = block["data"]["dur"] / 1000
                ampitude = block["data"]["amp"]
                fequencie = block["data"]["freq"]

                t = numpy.linspace(0, duration_in_seconds, int(sample_rate *duration_in_seconds), endpoint=False)  # Time array
                audio_arr.extend(ampitude * numpy.sin(2 * numpy.pi * fequencie * t))  # Sine wave formula

            else: # wenn Pause
                duration_in_seconds = block["data"]["dur"] / 1000
                audio_arr.extend(numpy.zeros(int(sample_rate * duration_in_seconds)))
        return numpy.float32(audio_arr)

############################################################################################
# Visualisiert die Ausgabe (Json Objekt)
############################################################################################
def plotOutput(ojson):
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches

    plt.figure().set_figwidth(20)

    amplitudes = []
    frequencies = []
    times = []
    times.append(0)
    for block in ojson["sequence"]:
        if block["type"] == "v":
            amplitudes.append(block["data"]["amp"])
            frequencies.append(block["data"]["freq"])
        else:
            amplitudes.append(0)
            frequencies.append(0)
            if len(times) > 0:
                plt.axvspan(times[-1], times[-1] + block["data"]["dur"], color="green", alpha=0.5)
            else:
                plt.axvspan(0, block["data"]["dur"], color="green", alpha=1)
        if len(times) == 0:
            times.append(block["data"]["dur"])
        else:
            times.append(times[-1] + block["data"]["dur"])
    times_block_mid = []
    times_block_mid.append(times[0] - 0 / 2)


    line1 =plt.hlines(amplitudes, times[:-1], times[1:], color="blue",label="Amplitude")
    plt.ylabel("Amplitude")
    plt.xlabel("Zeit in Millisekunden")
    plt.legend(loc="upper right")
    plt.twinx()
    line2=plt.hlines(frequencies, times[:-1], times[1:], color="red",label="Frequenz")
    plt.ylabel("Frequenz in Hertz")
    plt.legend(handles=[line1, line2])
    plt.savefig(getFilePath(name="Blocks",ext=".svg"))

############################################################################################
# Visualisiert Signal
############################################################################################
def plotOutputSignal(audio_arr):
    import numpy
    import matplotlib.pyplot as plt
    times = numpy.arange(0,len(audio_arr)/16,1/16)
    plt.figure().set_figwidth(20)
    plt.plot(times, audio_arr,label="Signal")
    plt.hlines([-1,1],0,times[-1],colors="red",label="Amplitudengrenzen")
    plt.xlabel("Zeit in Millisekunden")
    plt.legend()
    plt.savefig(getFilePath(name="Signal",ext=".svg"))

############################################################################################
# Speichert Signal als wav-Datei
############################################################################################
def saveAsWav(audio_arr, sr):
    import scipy.io.wavfile
    filename = getFilePath(name="",ext=".wav")
    scipy.io.wavfile.write(filename, sr, audio_arr)
    return filename


generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True) #Ausgabe in String umwandeln
generated_text = findResponse(generated_text) #Findet die Antwort in string
oJson = fixUnfinishedJson(generated_text) # Fixt die Antwort falls sie nicht fertig generiert wurde
plotOutput(oJson) #Visualisiert die Antowrt

audio_arr = generateSamples(oJson) #Gneriert Signal aus der Antwort
plotOutputSignal(audio_arr) #Visualisiert das Signal

# wav-Audio erstellen
wavFile = saveAsWav(audio_arr, 16000)

# Audio anzeigen
from IPython import display
display.display(display.Audio(wavFile))




NameError: name 'outputs' is not defined

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Generate an vibrotaktil pattern with following characteristics:", # instruction
        "irregular,discontinuous,pointy,soft", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 10000)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. 

### Instruction:
Generate an vibrotaktil pattern with following characteristics:

### Input:
discontinuous,regular,grainy,rough,annoying,agitating,mechanical,urgent,engine,animal,electric shock,beep,alarm,above intended threshold,below intended threshold

### Response:
{"sequence": [{"type": "p", "data": {"dur": 26.3125}}, {"type": "v", "data": {"amp": 0.999958499267207, "freq": 199.25035489714883, "form": "sine", "dur": 26.3125}}, {"type": "v", "data": {"amp": 0.9999650335746109, "freq": 199.2599406182493, "form": "sine", "dur": 26.3125}}, {"type": "v", "data": {"amp": 0.9999649422673486, "freq": 199.2599242677156, "form": "sine", "dur": 26.3125}}, {"type": "v", "data": {"amp": 0.9999650199066658, "freq": 199.2599200427073, "form": "sine", "dur": 26.3125}}, {"type": "v", "data": {"amp": 0.9999649570877915, "fre

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


26.3125}}, 

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasGemmStridedBatchedEx( handle, opa, opb, m, n, k, (void*)(&falpha), a, CUDA_R_16F, lda, stridea, b, CUDA_R_16F, ldb, strideb, (void*)(&fbeta), c, CUDA_R_16F, ldc, stridec, num_batches, CUDA_R_32F, CUBLAS_GEMM_DEFAULT_TENSOR_OP)`

### Evaluation

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

for data in dataset["test"]:
  input = ""
  if examples["sensationTags"][i] is not None:
    input = input + examples["sensationTags"][i] + ","
  if examples["emotionTags"][i] is not None:
    input = input + examples["emotionTags"][i] + ","
  if examples["metaphors"][i] is not None:
    input = input + examples["metaphors"][i] + ","
  if examples["usageExamples"][i] is not None:
    input = input + examples["usageExamples"][i]
  output = readJson(examples["id"][i])

  inputs = tokenizer(
  [
      alpaca_prompt.format(
          data["instruction"], # instruction
          train_dataset # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_length = 8000,max_time=600,use_cache = True) #LLama 3 unterstützt max 8000 Tokens
tokenizer.batch_decode(outputs)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous landmark in Paris, France. It is a wrought iron tower that was built in 1889 for the World's Fair. Standing at 324 meters tall, it is the tallest building in Paris and one of the most recognizable landmarks in the world.<|end_of_text|>"]

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
ID = "five beeps" #für das Speichern von Ergebnissen
VERSION = get_next_version(getFilePath())

prompt = "short beeps, six long breaks, ramping up"

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
       alpaca_prompt.format(
        "Generate an vibrotaktil pattern with following characteristics:", # instruction
        prompt,
        "", # output - leave this blank for generation!
    )

], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_length = 8000,max_time=360,use_cache = True) #LLama 3 unterstützt max 8000 Tokens #Abbrechen nach 10 Minuten
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate an vibrotaktil pattern with following characteristics:\n\n### Input:\nshort beeps, six long breaks, ramping up\n\n### Response:\n{"sequence": [{"type": "p", "data": {"dur": 51.19}}, {"type": "v", "data": {"amp": 0.507, "freq": 155.2, "form": "sine", "dur": 35.75}}, {"type": "v", "data": {"amp": 0.32, "freq": 161.8, "form": "sine", "dur": 26.06}}, {"type": "v", "data": {"amp": 0.201, "freq": 183.6, "form": "sine", "dur": 31.94}}, {"type": "v", "data": {"amp": 0.094, "freq": 185.2, "form": "sine", "dur": 63.62}}, {"type": "p", "data": {"dur": 109.19}}, {"type": "v", "data": {"amp": 0.507, "freq": 155.2, "form": "sine", "dur": 35.75}}, {"type": "v", "data": {"amp": 0.32, "freq": 161.8, "form": "sine", "dur": 26.06}}, {"type": "v", "data": {"amp": 0.201, "freq": 183.6, "form": "sine", "

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("/content/drive/MyDrive/vibviz/lora-alpaca-gguf", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.73 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:35<00:00,  2.98s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving /content/drive/MyDrive/vibviz/lora-alpaca-gguf/pytorch_model-00001-of-00004.bin...
Unsloth: Saving /content/drive/MyDrive/vibviz/lora-alpaca-gguf/pytorch_model-00002-of-00004.bin...
Unsloth: Saving /content/drive/MyDrive/vibviz/lora-alpaca-gguf/pytorch_model-00003-of-00004.bin...
Unsloth: Saving /content/drive/MyDrive/vibviz/lora-alpaca-gguf/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at /content/drive/MyDrive/vibviz/lora-alpaca-gguf into f16 GGUF format.
The output location will be .//content/drive/MyDrive/vibviz/lora-alpaca-gguf/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: lora-alpaca-gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 

RuntimeError: Unsloth: Quantization failed for .//content/drive/MyDrive/vibviz/lora-alpaca-gguf/unsloth.F16.gguf
You might have to compile llama.cpp yourself, then run this again.
You do not need to close this Python program. Run the following commands in a new terminal:
You must run this in the same folder as you're saving your model.
git clone --recursive https://github.com/ggerganov/llama.cpp
cd llama.cpp && make clean && make all -j
Once that's done, redo the quantization.

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>